In [3]:
import numpy as np

from PIL import Image, ImageDraw
import torch
import torchvision
from IPython.display import display

np.random.seed(42)

In [4]:
train_Images_folder = '/Users/abdulrehman/Desktop/SML/WIDER_train/images/'
test_Images_folder = '/Users/abdulrehman/Desktop/SML/WIDER_val/images/'
train_bbx = '/Users/abdulrehman/Desktop/SML/wider_face_split/wider_face_train_bbx_gt.txt'
test_bbx = '/Users/abdulrehman/Desktop/SML/wider_face_split/wider_face_val_bbx_gt.txt'
data_folder = '/Users/abdulrehman/Desktop/SML/Faces'

Need to remove occlusion
need to remove invalid images

In [8]:
def create_data_dict(bbx_path,images_path):
    data = {}
    with open(bbx_path, 'r') as file:
        while True:
            line = file.readline()[:-1]#reads the image path
            if not line: break
            faces = []
            n = int(file.readline())#reads the number of bounding boxes present in the image at Path(line)
            check_valid = 1
            im = Image.open(images_path+line)
            l,w,_ = np.array(im).shape
            if n>0:
                while n>0:
                    #reads bounding boxes values and types
                    v = np.fromstring(file.readline(), dtype = float, sep=' ')
                    if check_valid ==1:
                    #if not clear or not valid or contains heavy occlusion do not consider
                        if v[4]!=0 or v[7]==1 or v[8]==2:
                            check_valid = 0
                        area = v[2]*v[3]
                        if area<100:
                            check_valid = 0
                        faces.append([v[0]/w, v[1]/l, v[2]/w, v[3]/l])
                    n-=1
                if check_valid:
                    data[line]=np.array(faces)
                    print(data[line])
            else: line = file.readline()
    
    return data

In [9]:
train_data = create_data_dict(train_bbx,train_Images_folder)

# test_data = create_data_dict(test_bbx,test_Images_folder)

[[0.43847656 0.23826715 0.11914062 0.10758123]]
[[0.35253906 0.06843575 0.25683594 0.23673184]]
[[0.56933594 0.49348958 0.05566406 0.08072917]
 [0.66015625 0.38932292 0.04394531 0.078125  ]
 [0.6015625  0.35807292 0.03808594 0.05338542]
 [0.60253906 0.29166667 0.03613281 0.04817708]
 [0.44238281 0.3828125  0.04296875 0.06901042]
 [0.35253906 0.36588542 0.03222656 0.05598958]
 [0.36523438 0.49479167 0.04882812 0.07161458]
 [0.24414062 0.5390625  0.04101562 0.06901042]
 [0.47363281 0.54557292 0.046875   0.08463542]]
[[0.28710938 0.18951358 0.44433594 0.36765635]]
[[0.51953125 0.16       0.21386719 0.27333333]]
[[0.42871094 0.12880396 0.18066406 0.17975938]]
[[0.26660156 0.21289062 0.26953125 0.2421875 ]]
[[0.20214844 0.29882812 0.32226562 0.28710938]]
[[0.76269531 0.50065104 0.06152344 0.06445312]]
[[0.32421875 0.16666667 0.20507812 0.38541667]
 [0.52929688 0.30208333 0.1796875  0.34635417]]
[[0.34082031 0.13103953 0.16699219 0.16178624]]
[[0.09082031 0.11523929 0.10351562 0.08375315]
 [

[[0.44433594 0.15469974 0.08496094 0.08224543]]
[[0.39453125 0.10875332 0.08984375 0.16976127]]
[[0.56445312 0.1294964  0.13867188 0.29064748]
 [0.453125   0.49784173 0.12304688 0.21007194]
 [0.21289062 0.2676259  0.125      0.24460432]]
[[0.34570312 0.22395833 0.07617188 0.16666667]]
[[0.36914062 0.20572917 0.29492188 0.58333333]]
[[0.3984375  0.28385417 0.06835938 0.13541667]]
[[0.25976562 0.16550117 0.31445312 0.56410256]]
[[0.20507812 0.17149758 0.06445312 0.13285024]
 [0.69726562 0.14009662 0.06445312 0.16183575]]
[[0.35546875 0.21752266 0.20703125 0.28600201]]
[[0.18359375 0.35202086 0.07421875 0.12255541]
 [0.6953125  0.39895698 0.07617188 0.12255541]]
[[0.35351562 0.30088496 0.0546875  0.15929204]
 [0.47460938 0.28672566 0.05273438 0.14513274]
 [0.62304688 0.28318584 0.05273438 0.14159292]]
[[0.20800781 0.14648438 0.24902344 0.20898438]]
[[0.60351562 0.10344828 0.08203125 0.14854111]
 [0.76367188 0.14323607 0.08789062 0.18567639]
 [0.140625   0.0530504  0.0625     0.25464191]]


[[0.21582031 0.25759577 0.04492188 0.07926024]
 [0.28808594 0.28137384 0.04003906 0.07926024]
 [0.70605469 0.33817701 0.04003906 0.07926024]
 [0.42089844 0.31307794 0.04003906 0.07793923]
 [0.47167969 0.35535007 0.04492188 0.06340819]
 [0.50292969 0.29062087 0.02929688 0.06208719]
 [0.51757812 0.22060766 0.0390625  0.08058124]]
[[0.45898438 0.29154519 0.1328125  0.25655977]]
[[0.34375    0.00877193 0.19726562 0.44736842]]
[[0.62011719 0.22148541 0.03710938 0.06763926]
 [0.71484375 0.22413793 0.0390625  0.06100796]
 [0.74316406 0.26923077 0.01757812 0.03846154]
 [0.59375    0.29310345 0.015625   0.02387268]
 [0.86132812 0.22148541 0.02539062 0.05702918]]
[[0.19433594 0.08333333 0.04980469 0.09356725]
 [0.30175781 0.34356725 0.03320312 0.06871345]
 [0.48046875 0.27631579 0.0390625  0.07163743]
 [0.66308594 0.3128655  0.03515625 0.0628655 ]
 [0.79785156 0.25       0.03808594 0.07163743]
 [0.89648438 0.24269006 0.04101562 0.0745614 ]
 [0.875      0.26608187 0.05273438 0.04678363]]
[[0.2128

[[0.10839844 0.10183486 0.11621094 0.15229358]
 [0.23925781 0.18348624 0.13769531 0.1559633 ]
 [0.31640625 0.06880734 0.1328125  0.13302752]
 [0.41601562 0.1266055  0.14355469 0.16972477]
 [0.54492188 0.1        0.1328125  0.20917431]]
[[0.10742188 0.3318649  0.06445312 0.13509545]
 [0.31640625 0.30249633 0.05664062 0.12041116]
 [0.47460938 0.38472834 0.05859375 0.13803231]
 [0.6171875  0.37885463 0.06445312 0.13509545]
 [0.8046875  0.27606461 0.0625     0.14096916]]
[[0.30078125 0.36200257 0.02636719 0.04749679]
 [0.31152344 0.42233633 0.03027344 0.05134788]
 [0.35351562 0.41463415 0.02832031 0.0449294 ]
 [0.37890625 0.37098845 0.02441406 0.04621309]
 [0.41503906 0.37483954 0.01953125 0.03851091]
 [0.43457031 0.40821566 0.02246094 0.03722721]
 [0.46777344 0.41591784 0.03710938 0.05776637]
 [0.49316406 0.37869063 0.02539062 0.04621309]
 [0.54492188 0.39922978 0.02832031 0.05391528]
 [0.5703125  0.35943517 0.02539062 0.03979461]
 [0.4609375  0.49422336 0.03515625 0.06546855]
 [0.5751953

[[0.60644531 0.68281938 0.04980469 0.0866373 ]
 [0.59277344 0.24522761 0.05078125 0.08370044]
 [0.69042969 0.05580029 0.05371094 0.09397944]
 [0.45214844 0.06754772 0.05175781 0.09838473]
 [0.45703125 0.34801762 0.046875   0.09838473]
 [0.28320312 0.34948605 0.046875   0.07782673]
 [0.26464844 0.1174743  0.04785156 0.09397944]
 [0.15917969 0.10279001 0.05078125 0.09251101]]
[[0.01171875 0.03258656 0.3046875  0.90835031]
 [0.31835938 0.00407332 0.3046875  0.89613035]]
[[0.1953125  0.32081911 0.11328125 0.17064846]
 [0.171875   0.10011377 0.08789062 0.15699659]
 [0.25390625 0.1069397  0.0703125  0.12059158]
 [0.38671875 0.0887372  0.07617188 0.11831627]
 [0.46679688 0.22298066 0.06640625 0.10466439]
 [0.56054688 0.1592719  0.0703125  0.12741752]
 [0.6328125  0.16609784 0.09179688 0.13651877]
 [0.73828125 0.17292378 0.11523438 0.16837315]]
[[0.42773438 0.27083333 0.17578125 0.2578125 ]]
[[0.1484375  0.5136742  0.125      0.13793103]
 [0.21289062 0.35671819 0.11328125 0.11414982]
 [0.32421

...

[[0.4296875  0.13872832 0.21972656 0.28323699]]
[[0.63476562 0.09179688 0.10546875 0.13085938]]
[[0.65820312 0.06442167 0.12695312 0.23718887]
 [0.26171875 0.40995608 0.14257812 0.21961933]]
[[0.29296875 0.11805556 0.09375    0.26041667]
 [0.59375    0.07986111 0.10546875 0.25      ]]
[[0.54003906 0.1208577  0.20507812 0.18908382]]
[[0.55273438 0.13802083 0.1875     0.32552083]]
[[0.2265625  0.35677083 0.078125   0.140625  ]
 [0.44335938 0.34114583 0.08203125 0.14322917]
 [0.72460938 0.40885417 0.07617188 0.13541667]]
[[0.79003906 0.11419753 0.05078125 0.23148148]
 [0.40917969 0.11419753 0.04492188 0.23148148]
 [0.09765625 0.10802469 0.04492188 0.23765432]]
[[0.37402344 0.10784314 0.21777344 0.2753268 ]]
[[0.29199219 0.11502516 0.22265625 0.23436377]]
[[0.1640625  0.33764368 0.26757812 0.21623563]]
[[0.625      0.03119584 0.12304688 0.29462738]]
[[0.75585938 0.11127379 0.05859375 0.12591508]
 [0.49609375 0.18740849 0.0546875  0.12005857]
 [0.2734375  0.14934114 0.0625     0.0966325 ]]


[[0.50390625 0.18763326 0.33007812 0.30277186]
 [0.08007812 0.20469083 0.2578125  0.23383085]
 [0.34082031 0.06609808 0.16308594 0.16773276]
 [0.30566406 0.56218905 0.43164062 0.20326937]]
[[0.3125     0.0626151  0.39941406 0.55801105]]
[[0.24902344 0.13300142 0.08007812 0.08036984]
 [0.53613281 0.08036984 0.08886719 0.09743954]]
[[0.3828125  0.24798387 0.24414062 0.32459677]]
[[0.41992188 0.09722222 0.27929688 0.70486111]]
[[0.59960938 0.11939268 0.13769531 0.14423741]]
[[0.29589844 0.17933723 0.30175781 0.22417154]]
[[0.36328125 0.15010142 0.15234375 0.13049358]]
[[0.39160156 0.1732899  0.22558594 0.21824104]]
[[0.6640625  0.10164425 0.33398438 0.70254111]
 [0.1796875  0.0328849  0.35742188 0.7264574 ]]
[[0.18945312 0.23987539 0.09765625 0.28037383]
 [0.27929688 0.47975078 0.0703125  0.15264798]
 [0.51757812 0.14330218 0.12304688 0.28971963]
 [0.70117188 0.34579439 0.11328125 0.23676012]]
[[0.84765625 0.36176471 0.0703125  0.10882353]]
[[0.34375    0.05273438 0.33398438 0.4609375 ]]


[[0.44335938 0.09775429 0.140625   0.28533686]]
[[0.140625   0.14985591 0.12890625 0.25072046]]
[[0.3046875  0.23177083 0.16992188 0.25520833]]
[[0.3046875  0.15494792 0.04980469 0.08984375]
 [0.22265625 0.53515625 0.06054688 0.09895833]
 [0.42382812 0.19791667 0.05175781 0.08072917]
 [0.48828125 0.46484375 0.0546875  0.09765625]
 [0.58300781 0.17708333 0.04882812 0.08072917]
 [0.73144531 0.18098958 0.05273438 0.09114583]]
[[0.45507812 0.07583333 0.20117188 0.19916667]]
[[0.07421875 0.18832891 0.09570312 0.22281167]
 [0.2109375  0.25729443 0.10351562 0.22015915]
 [0.45117188 0.20954907 0.08398438 0.17771883]
 [0.59765625 0.19363395 0.11328125 0.19098143]
 [0.73046875 0.16976127 0.09375    0.19363395]
 [0.86328125 0.16710875 0.09375    0.20689655]]
[[0.24414062 0.22887701 0.265625   0.38074866]
 [0.52929688 0.20534759 0.25195312 0.35935829]]
[[0.28125    0.09814324 0.16015625 0.34217507]]
[[0.39550781 0.16032211 0.171875   0.16398243]]
[[0.20214844 0.17604618 0.04101562 0.08225108]
 [0.

[[0.22949219 0.17331499 0.41796875 0.39202201]]
[[0.30566406 0.16408935 0.23242188 0.23969072]]
[[0.34570312 0.1328125  0.24316406 0.1875    ]]
[[0.36621094 0.17154812 0.32519531 0.31380753]]
[[0.40039062 0.12868633 0.22753906 0.19101877]]
[[0.15429688 0.04308202 0.08496094 0.10521955]
 [0.36523438 0.15824358 0.08007812 0.09362055]
 [0.53515625 0.15824358 0.08007812 0.08782104]
 [0.8046875  0.11350456 0.08007812 0.09527755]]
[[0.35546875 0.14072633 0.21777344 0.16407263]
 [0.65625    0.15629053 0.11132812 0.14072633]
 [0.83789062 0.23605707 0.09667969 0.16018158]]
[[0.25488281 0.13257066 0.30566406 0.28129206]]
[[0.40625    0.16223232 0.4296875  0.38286827]]
[[0.3359375  0.17604618 0.28613281 0.3008658 ]]
[[0.15332031 0.1640178  0.55566406 0.49459631]]
[[0.22851562 0.16129032 0.18554688 0.40762463]
 [0.06835938 0.30498534 0.10742188 0.26099707]
 [0.4375     0.68914956 0.08007812 0.19648094]
 [0.81054688 0.23753666 0.17773438 0.3372434 ]]
[[0.33398438 0.165625   0.29492188 0.3296875 ]]


[[0.15527344 0.234375   0.58984375 0.54361979]]
[[0.1875     0.19902518 0.58886719 0.66206336]]
[[0.046875   0.06404658 0.4296875  0.87918486]]
[[0.4453125 0.13125   0.171875  0.4      ]]
[[0.34667969 0.1248366  0.41992188 0.38627451]]
[[0.39453125 0.18899676 0.27929688 0.25372168]]
[[0.27148438 0.16581446 0.41992188 0.44704164]]
[[0.33007812 0.10541728 0.13085938 0.23426061]]
[[0.32617188 0.09150327 0.28320312 0.27908497]]
[[0.26953125 0.11906311 0.15820312 0.12687053]
 [0.64550781 0.09954457 0.16113281 0.13467794]]
[[0.30078125 0.14276402 0.28027344 0.22229465]]
[[0.41015625 0.15234375 0.27050781 0.30625   ]]
[[0.3515625  0.0779681  0.16113281 0.14648553]]
[[0.1171875  0.22258274 0.49121094 0.44970798]]
[[0.27636719 0.27792208 0.48535156 0.40844156]]
[[0.33789062 0.28181818 0.31445312 0.25844156]]
[[0.29003906 0.15833974 0.40722656 0.4319754 ]]
[[0.31347656 0.17620751 0.25976562 0.28533095]]
[[0.36816406 0.18306189 0.22558594 0.18697068]]
[[0.42480469 0.11728395 0.15527344 0.14609053

In [17]:
from shutil import copyfile
from pathlib import Path

def create_dataset(data,dataset_type):
    if dataset_type == 'train':
        Images_folder = train_Images_folder
    else:
        Images_folder = test_Images_folder
    
    images_path = Path(data_folder+'/Images/'+dataset_type)
    images_path.mkdir(parents=True, exist_ok=True)
    labels_path = Path(data_folder+'/labels/'+dataset_type)
    labels_path.mkdir(parents=True, exist_ok=True)
    
    name = 1
    for path in data:
        copyfile(Images_folder+path, images_path/(str(name)+'.jpg'))
        with open(labels_path/(str(name)+'.txt'),'w') as out_file:
            for bbx in data[path]:
                x,y,w,h = [float(i) for i in bbx]
                out_file.write('1 '+str(x+w/2)+' '+str(y+h/2)+' '+str(w)+' '+str(h)+'\n')
        name+=1

In [19]:
create_dataset(train_data,'train')

In [18]:
create_dataset(test_data,'test')

In [24]:
def draw_bounding_boxes(im, bbxs,color='red',width=3):
    draw = ImageDraw.Draw(im)
    l,w,_ = [float(i) for i in np.array(im).shape]
    for box in bbxs:
        x0 = (box[1]-box[3]/2)*w
        y0 = (box[2]-box[4]/2)*l
        x1 = (box[1]+box[3]/2)*w
        y1 = (box[2]+box[4]/2)*l
        draw.rectangle([x0,y0,x1,y1], fill=None, outline=color, width=width)
    return(im)

image_name = '1600'
bbxs = []
with open(data_folder+'/labels/train/'+image_name+'.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        line = np.fromstring(line, dtype=float, sep=' ')
        
        bbxs.append(line)

im = Image.open(data_folder+'/Images/train/'+image_name+'.jpg')
im = draw_bounding_boxes(im,bbxs)
im.show()